In [25]:
import numpy as np
import pandas as pd
import pyodbc

In [26]:
conn = pyodbc.connect(driver = "SQL Server",
                      server='TEST',
                      database='TEST',
                      uid='TEST',pwd='Pass)

In [27]:
############################ CUSTOMER ##########################################
query_customer= """SELECT CUST_ID, ADR_STREET, ADR_SUB_DISTRICT, ADR_DISTRICT, ADR_PROVINCE, ADR_ZIP_CODE
FROM sas..CLM_MST_CUSTOMER_CAR_BASE
WHERE CUST_ID IS NOT NULL AND PDPA1 = 'Y' AND PDPA2 = 'Y' AND PDPA3 ='Y' AND PDPA4 = 'Y' AND PDPA5 = 'Y' AND CMS_MOBILE_TOKEN IS NOT NULL
"""

df_customer = pd.read_sql_query(query_customer, conn)
df_customer.head(5)

D:\Users\kasidits\AppData\Local\Temp\ipykernel_4580\3451138852.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_customer = pd.read_sql_query(query_customer, conn)


,MAIN_IDENTIFIER_NO,ADR_STREET,ADR_SUB_DISTRICT,ADR_DISTRICT,ADR_PROVINCE,ADR_ZIP_CODE
0,1003172237,None,หมอนนาง,พนัสนิคม,ชลบุรี,20140
1,2000773386,None,อาจสามารถ,อาจสามารถ,ร้อยเอ็ด,45160
2,1002177448,None,คลองเจ็ด,คลองหลวง,ปทุมธานี,12120
3,1001029658,ทวีวัฒนา,ศาลาธรรมสพน์,ทวีวัฒนา,กรุงเทพมหานคร,10170
4,2001710212,None,ศาลาลอย,ท่าเรือ,พระนครศรีอยุธยา,13130


In [28]:
df_lat_lon = pd.read_csv("D:/Users/kasidits/OneDrive - Home Product Center Public Company Limited/Code/Python Code/CSV/20240607 THAILAND POSTCODE.csv")
df_lat_lon.head(5)

,PostCode,TambonID,หมายเหตุ,TambonThaiShort,DistrictThaiShort,ProvinceThai,ภูมิภาคอย่างเป็นทางการ,กรุงเทพปริมณฑลต่างจังหวัด,Unnamed: 8,Unnamed: 9,province_eng,region,Latitude,Longitude
0,10100,101301,NaN,จักรวรรดิ,สัมพันธวงศ์,กรุงเทพมหานคร,ภาคกลาง,กรุงเทพมหานครและปริมณฑล,NaN,NaN,Bangkok,BANGKOK,13.741,100.504
1,10100,100804,NaN,บ้านบาตร,ป้อมปราบศัตรูพ่าย,กรุงเทพมหานคร,ภาคกลาง,กรุงเทพมหานครและปริมณฑล,NaN,NaN,Bangkok,BANGKOK,13.752,100.507
2,10100,100801,NaN,ป้อมปราบ,ป้อมปราบศัตรูพ่าย,กรุงเทพมหานคร,ภาคกลาง,กรุงเทพมหานครและปริมณฑล,NaN,NaN,Bangkok,BANGKOK,13.743,100.514
3,10100,101303,NaN,ตลาดน้อย,สัมพันธวงศ์,กรุงเทพมหานคร,ภาคกลาง,กรุงเทพมหานครและปริมณฑล,NaN,NaN,Bangkok,BANGKOK,13.734,100.513
4,10100,100802,NaN,วัดเทพศิรินทร์,ป้อมปราบศัตรูพ่าย,กรุงเทพมหานคร,ภาคกลาง,กรุงเทพมหานครและปริมณฑล,NaN,NaN,Bangkok,BANGKOK,13.749,100.512


In [29]:
df_customer_lat_lon_full = df_customer.merge(df_lat_lon,
                                        left_on
                                        =['ADR_SUB_DISTRICT', 'ADR_DISTRICT', 'ADR_PROVINCE'],
                                        right_on
                                        =['TambonThaiShort', 'DistrictThaiShort', 'ProvinceThai'])
df_customer_lat_lon = df_customer_lat_lon_full[['CUST_ID','Latitude','Longitude','ADR_SUB_DISTRICT', 'ADR_DISTRICT', 'ADR_PROVINCE']]
df_customer_lat_lon.head(5)

,MAIN_IDENTIFIER_NO,Latitude,Longitude,ADR_SUB_DISTRICT,ADR_DISTRICT,ADR_PROVINCE
0,1003172237,13.346,101.257,หมอนนาง,พนัสนิคม,ชลบุรี
1,1085868328,13.346,101.257,หมอนนาง,พนัสนิคม,ชลบุรี
2,1093867986,13.346,101.257,หมอนนาง,พนัสนิคม,ชลบุรี
3,2001795714,13.346,101.257,หมอนนาง,พนัสนิคม,ชลบุรี
4,1092143234,13.346,101.257,หมอนนาง,พนัสนิคม,ชลบุรี


In [33]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance_km = R * c
    return distance_km
 
#BJB 13.938555514184458, 100.39315546373766
#BJK 13.713224104121844, 100.58312849790039
store_lat = 13.713224104121844
store_lon = 100.58312849790039
pin = 'BJK'
 
df_customer_lat_lon['distance_km'] = haversine_distance(
    store_lat, store_lon,
    df_customer_lat_lon['Latitude'], df_customer_lat_lon['Longitude']
)
 
radius_km = 10
df2 = df_customer_lat_lon[df_customer_lat_lon['distance_km'] <= radius_km]
df2.to_excel(f'D:/Users/kasidits/OneDrive - Home Product Center Public Company Limited/Code/Python Code/CSV/df_customer_lat_lon_2kma_{pin}.xlsx')

D:\Users\kasidits\AppData\Local\Temp\ipykernel_4580\3340244498.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_customer_lat_lon['distance_km'] = haversine_distance(
